# Congestion Charges - Medium

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@192.168.31.31:15432/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 60)


 ········


In [2]:
camera = pd.read_sql_table('camera', engine)
keeper = pd.read_sql_table('keeper', engine)
vehicle = pd.read_sql_table('vehicle', engine)
image = pd.read_sql_table('image', engine)
permit = pd.read_sql_table('permit', engine)

## 1.
List the owners (name and address) of Vehicles caught by camera 1 or 18 without duplication.

In [3]:
(keeper.rename(columns={'id': 'keeper'})
 .merge(vehicle, on='keeper')
 .merge(image.loc[image['camera'].isin([1, 18])], 
        left_on='id', right_on='reg')
 [['name', 'address']].drop_duplicates())

,name,address
0,"Ambiguous, Arthur",Absorption Ave.
6,"Inconspicuous, Iain",Interception Rd.
8,"Strenuous, Sam",Surjection Street


## 2.
Show keepers (name and address) who have more than 5 vehicles.

In [4]:
(keeper.rename(columns={'id': 'keeper'})
 .merge(vehicle, on='keeper')
 .groupby(['name', 'address'])['keeper'].count()
 .reset_index().query('keeper>5')
 [['name', 'address']])

,name,address
0,"Ambiguous, Arthur",Absorption Ave.
4,"Inconspicuous, Iain",Interception Rd.


## 3.
For each vehicle show the number of current permits (suppose today is the 1st of Feb 2007). The list should include the vehicle.s registration and the number of permits. Current permits can be determined based on charge types, e.g. for weekly permit you can use the date after 24 Jan 2007 and before 02 Feb 2007.

In [6]:
from pandas.tseries.offsets import DateOffset

t = permit.assign(edate=permit['sdate'])
charge_daily = t['chargetype']=='Daily'
t.loc[charge_daily, 'edate'] = t.loc[charge_daily, 'sdate'] + DateOffset(days=1)
charge_weekly = t['chargetype']=='Weekly'
t.loc[charge_weekly, 'edate'] = t.loc[charge_weekly, 'sdate'] + DateOffset(weeks=1)
charge_monthly = t['chargetype']=='Monthly'
t.loc[charge_monthly, 'edate'] = t.loc[charge_monthly, 'sdate'] + DateOffset(months=1)
charge_yearly = t['chargetype']=='Annual'
t.loc[charge_yearly, 'edate'] = t.loc[charge_yearly, 'sdate'] + DateOffset(years=1)

(t.loc[(t['sdate']<='2007-02-01') & (t['edate']>='2007-02-01')]
 .groupby('reg').agg(n_permit=pd.NamedAgg(column='chargetype', aggfunc='count'))
 .reset_index().sort_values('reg'))

,reg,n_permit
0,SO 02 DSP,1
1,SO 02 DTP,1
2,SO 02 JSP,1
3,SO 02 KSP,1
4,SO 02 KTP,1
5,SO 02 QSP,1
6,SO 02 RSP,1


## 4.
Obtain a list of every vehicle passing camera 10 on 25th Feb 2007. Show the time, the registration and the name of the keeper if available.

In [7]:
(vehicle.merge(keeper.rename(columns={'id': 'keeper'}),
               on='keeper')
 .merge(image.loc[(image['whn'].between('2007-02-25', '2007-02-25 23:59:59')) &
                  (image['camera']==10)], left_on='id', right_on='reg')
 [['reg', 'whn', 'name']])

,reg,whn,name
0,SO 02 CSP,2007-02-25 07:45:11,"Ambiguous, Arthur"
1,SO 02 ESP,2007-02-25 18:08:40,"Ambiguous, Arthur"


## 5.
List the keepers who have more than 4 vehicles and one of them must have more than 2 permits. The list should include the names and the number of vehicles.

In [8]:
# keepers who have more than 2 permits
p = (permit.groupby('reg')
     .agg(npermit=pd.NamedAgg(column='chargetype', aggfunc='count'))
     .reset_index())
t = (vehicle.loc[vehicle['id'].isin(p.loc[p['npermit']>2, 'reg'])])

(vehicle.merge(keeper.loc[keeper['id'].isin(t['keeper'])]
               .rename(columns={'id': 'keeper'}),
               on='keeper')
 .groupby('name')
 .agg(n_vehicle=pd.NamedAgg(column='id', aggfunc='count'))
 .reset_index()[['name', 'n_vehicle']]
 .query('n_vehicle>4'))

,name,n_vehicle
0,"Inconspicuous, Iain",7
